In [1]:
import os
import re
import geonamescache
import pandas as pd
from unidecode import unidecode
from operator import itemgetter

os.path.abspath('.')

headlines_file = open('data/headlines.txt')
# headlines_file = open('data/problematic.txt')
headlines = headlines_file.read().splitlines()
headlines_file.close()
# for headline in headlines:
#    print(headline)

gc = geonamescache.GeonamesCache()
cities1 = gc.get_cities()
ascii_cities = [city['geonameid'] for city in cities1.values() if city['name'] == unidecode(city['name'])]
names = {cities1[str(id)]['name']: cities1[str(id)]['geonameid'] for id in ascii_cities}
accented_cities = [city['geonameid'] for city in cities1.values() if city['name'] != unidecode(city['name'])]
alternative_names = {unidecode(cities1[str(id)]['name']): cities1[str(id)]['geonameid'] for id in accented_cities}
names.update(alternative_names)
alternative_names = {unidecode(cities1[str(id)]['name']): cities1[str(id)]['name'] for id in accented_cities}
del names['Of']
del names['Come']

def find_city(city, pattern, string):
    found = re.search(city_pattern, string)
    if found:
        without_city = re.sub(city_pattern, '', string).strip()
        without_city = re.sub(' {2,}', ' ', without_city)
        # print('City ', city, " is found in '", string, "'", sep='')
        return without_city
    return None


def prepare_cre(city):
    return re.compile(r'\b' + city + r'\b', re.ASCII | re.IGNORECASE)


headlines_without_cities = []
headlines_without_found_city = set()

city_candidates = dict()
for city in names.keys():
    city_pattern = prepare_cre(city)
    for headline in headlines:
        found = find_city(city, city_pattern, headline)
        if found:
            headlines_without_cities.append(found)
            if headline not in city_candidates:
                city_candidates[headline] = [city]
            else:
                city_candidates[headline].append(city)

df = pd.DataFrame(columns=['Heading', 'City', 'Country code'])
for i, headline in enumerate(city_candidates):
    cities = city_candidates[headline]
    cities.sort(key=lambda s: len(s), reverse=True)
    cities_list = gc.get_cities_by_name(cities[0])
    if not cities_list:
        cities_list = gc.get_cities_by_name(alternative_names[cities[0]])
    print(i, '. ', headline, sep='', end='')
    sorted_by_population = []
    for city in cities_list:
        sorted_by_population.append((int(list(city.keys())[0]), list(city.values())[0]['population']))
    sorted_by_population.sort(key=itemgetter(1), reverse=True)
    found_city = cities1[str(sorted_by_population[0][0])]
    df.loc[i] = [headline, found_city['name'], found_city['countrycode']]
    print(':', found_city['name'], ':', found_city['countrycode'])

df.to_csv("data/cities.csv", index=False)

# print(set(headlines) - set(city_candidates.keys()))

0. Authorities are Worried about the Spread of Norovirus in Dubai: Dubai : AE
1. West Nile Virus Symptoms Spread all over Yerevan: Yerevan : AM
2. Tuberculosis Hits Luanda: Luanda : AO
3. Outbreak of Zika in Santa Elena: Santa Elena : EC
4. Will Hepatitis B vaccine help La Paz?: La Paz : BO
5. Authorities are Worried about the Spread of Norovirus in Buenos Aires: Buenos Aires : AR
6. Hepatitis E re-emerges in Santa Rosa: Santa Rosa : PH
7. Bronchitis Symptoms Spread all over Santa Fe: Santa Fe : AR
8. Zika spreads to San Luis Potosi: San Luis Potosí : MX
9. San Juan reports 1st U.S. Zika-related death amid outbreak: San Juan : AR
10. Rumors about Hepatitis D Spreading in San Juan Capistrano have been Refuted: San Juan Capistrano : US
11. Zika Virus Reaches San Francisco: San Francisco : US
12. More Zika patients reported in Bella Vista: Bella Vista : DO
13. Will the Mad Cow Outbreak Reach Vienna?: Vienna : AT
14. Rumors about Hepatitis D spreading in Albany have been refuted: Albany : 

145. More people in Nadi are infected with HIV every year: Nadi : FJ
146. Rotavirus Vaccine is now Required in Labasa: Labasa : FJ
147. Will MCD vaccine help Strasbourg?: Strasbourg : FR
148. Scientists in Paris to look for answers: Paris : FR
149. Rumors about HIV spreading in Montpellier have been refuted: Montpellier : FR
150. Mad Cow Disease Detected in Lyon: Lyon : FR
151. Hepatitis D has Arrived in Grenoble: Grenoble : FR
152. More Livestock in Fontainebleau are infected with Mad Cow Disease: Fontainebleau : FR
153. Saint Croix under Zika threat: Saint Croix : VI
154. Sick Livestock Leads to Serious Trouble for Belfort: Belfort : FR
155. Gonorrhea has Arrived in Avon Lake: Avon Lake : US
156. Hepatitis E has not Left Libreville: Libreville : GA
157. Could Zika Reach New York City?: New York City : US
158. West Nile Virus re-emerges in Wellington: Wellington : NZ
159. Hepatitis A re-emerges in Stamford: Stamford : US
160. Southampton Patient in Critical Condition after Contracting

293. Lung Cancer Rates Increasing in Moscow: Moscow : RU
294. Mecca tests new cure for Chikungunya: Mecca : SA
295. Rumors about Rabies Spreading in Khartoum have been Refuted: Khartoum : SD
296. Spike of Dengue Cases in Stockholm: Stockholm : SE
297. Zika cases in Singapore reach 393: Singapore : SG
298. Will Swine Flu vaccine help Ljubljana?: Ljubljana : SI
299. Cholera outbreak in Dakar: Dakar : SN
300. More Zika cases in Soyapango: Soyapango : SV
301. Zika Reported in Ilopango: Ilopango : SV
302. Hepatitis C re-emerges in Damascus: Damascus : SY
303. Nha Trang Zika Outbreak: Nha Trang : VN
304. Zika spreads to Kamphaeng Phet: Kamphaeng Phet : TH
305. Zika arrives in Udon Thani: Udon Thani : TH
306. Zika case reported in Phetchabun: Phetchabun : TH
307. Thailand-Zika Virus in Bangkok: Bangkok : TH
308. More Zika patients reported in Chanthaburi: Chanthaburi : TH
309. Zika case recorded in Tunapuna »: Tunapuna : TT
310. Sangre Grande Residents Recieve Gonorrhea vaccine: Sangre Grande

436. Rumors about Hepatitis E Spreading in Morristown have been Refuted: Morristown : US
437. Nashville takes precautions against Zika: Nashville : US
438. Spike of Rhinovirus Cases in Sevierville: Sevierville : US
439. Zika case reported in Abilene: Abilene : US
440. Zika symptoms spotted in Arlington: Arlington : US
441. More Zika patients reported in Austin: Austin : US
442. Outbreak of Zika in Corpus Christi: Corpus Christi : US
443. More Zika patients reported in Fort Worth: Fort Worth : US
444. Frisco Woman Tests Positive For Zika Virus: Frisco : US
445. Flu outbreak in Galveston, Texas: Galveston : US
446. Zika Concerns are Spreading in Houston: Houston : US
447. Zika spreads to Lewisville: Lewisville : US
448. More Zika patients reported in Mcallen: McAllen : US
449. New medicine wipes out Dengue in Saginaw: Saginaw : US
450. Outbreak of Zika in Stephenville: Stephenville : US
451. More Zika patients reported in Waco: Waco : US
452. Zika Outbreak in Weslaco: Weslaco : US
453. H

571. HIV has Arrived in Tacoma: Tacoma : US
572. The Spread of Dengue in Yakima has been Confirmed: Yakima : US
573. Measles Vaccine is now Required in Wailuku: Wailuku : US
574. Stony Brook University experts discuss Zika virus: University : US
575. Fort Hood Reports its First Zika Patient: Fort Hood : US
576. Montevideo Encounters Severe Symptoms of Rotavirus: Montevideo : UY
577. Caracas patient dies: Caracas : VE
578. Hospitals in Hanoi fill up with Zika patients: Hanoi : VN
579. Chlamydia Exposure in Sanaa: Sanaa : YE
580. Tuberculosis re-emerges in Silver Springs: Springs : ZA
581. Malaria is Spreading in Johannesburg: Johannesburg : ZA
582. Johannesburg Patient in Critical Condition after Contracting Pneumonia: Johannesburg : ZA
583. Mpika authorities confirmed the spread of Chikungunya: Mpika : ZM
584. More people in Kitwe are infected with Respiratory Syncytial Virus every year: Kitwe : ZM
585. Harare is infested with Pneumonia: Harare : ZW
586. More Zika patients reported in 